
#### Aim: To predict whether a customer is going to churn a telco service
##### Steps:
    - Importing and exploring data
    - Data preprocessing
    - 

In [360]:
# Import the important Library
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict
from sklearn import metrics
from pylab import rcParams

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

## 1. Importing and exploring data

In [361]:
# Load both the train and test
train = pd.read_csv('data_train.csv')
test = pd.read_csv('data_test.csv')

# concatenate the two sets and add and indicator column
#data = pd.concat([ train, test])
data = pd.concat([train.assign(ind="train"), test.assign(ind="test")])

# use the following line to separate the sets back after preprocessing
#  test, train = df[df["ind"].eq("test")], df[df["ind"].eq("train")]
print(data.shape)
data.head()

(7043, 51)


Customer ID  Count.x  Gender   Age Under 30 Senior Citizen Married  \
0  0650-BWOZN      1.0    Male  37.0       No             No     NaN   
1  0562-FGDCR      1.0     NaN  54.0       No             No      No   
2  6688-UZPWD      1.0    Male   NaN      Yes             No     NaN   
3  2905-KFQUV      1.0    Male  23.0      Yes             No      No   
4  9720-JJJOR      1.0  Female  41.0       No            NaN     Yes   

  Dependents  Number of Dependents  Count.y  ... Paperless Billing  \
0         No                   0.0      NaN  ...               Yes   
1         No                   0.0      1.0  ...                No   
2        Yes                   NaN      1.0  ...               NaN   
3         No                   0.0      1.0  ...               NaN   
4         No                   0.0      1.0  ...                No   

    Payment Method Monthly Charge  Total Charges Total Refunds  \
0  Bank Withdrawal          85.15        3030.60           0.0   
1              NaN          35.90          35.90           0.0   
2  Bank Withdrawal          20.05         746.75           NaN   
3              NaN          74.65         544.55           0.0   
4      Credit Card          25.80        1911.50           0.0   

   Total Extra Data Charges  Total Long Distance Charges  Total Revenue  \
0                       0.0                       892.85        3923.45   
1                       0.0                         0.00          35.90   
2                       0.0                      1435.70        2182.45   
3                      10.0                          NaN         705.61   
4                       NaN                      1164.16        3075.66   

   Churn Category    ind  
0      Competitor  train  
1      Competitor  train  
2        No Churn  train  
3      Competitor  train  
4        No Churn  train  

[5 rows x 51 columns]

In [362]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7043 entries, 0 to 1408
Data columns (total 51 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Customer ID                        7043 non-null   object 
 1   Count.x                            5393 non-null   float64
 2   Gender                             5393 non-null   object 
 3   Age                                5393 non-null   float64
 4   Under 30                           5393 non-null   object 
 5   Senior Citizen                     5393 non-null   object 
 6   Married                            5393 non-null   object 
 7   Dependents                         5393 non-null   object 
 8   Number of Dependents               5393 non-null   float64
 9   Count.y                            4713 non-null   float64
 10  Country                            4703 non-null   object 
 11  State                              4704 non-null   objec

In [363]:
data.dtypes

Customer ID                           object
Count.x                              float64
Gender                                object
Age                                  float64
Under 30                              object
Senior Citizen                        object
Married                               object
Dependents                            object
Number of Dependents                 float64
Count.y                              float64
Country                               object
State                                 object
City                                  object
Zip Code                             float64
Lat Long                              object
Latitude                             float64
Longitude                            float64
ID                                   float64
Population                           float64
Satisfaction Score                   float64
Count                                float64
Quarter                               object
Referred a

In [364]:
data.describe()

Count.x          Age  Number of Dependents  Count.y      Zip Code  \
count   5393.0  5393.000000           5393.000000   4713.0   4698.000000   
mean       1.0    46.747821              0.479510      1.0  93483.130268   
std        0.0    16.803802              0.971972      0.0   1861.284861   
min        1.0    19.000000              0.000000      1.0  90001.000000   
25%        1.0    33.000000              0.000000      1.0  92083.000000   
50%        1.0    46.000000              0.000000      1.0  93516.000000   
75%        1.0    60.000000              0.000000      1.0  95337.000000   
max        1.0    80.000000              9.000000      1.0  96150.000000   

          Latitude    Longitude           ID     Population  \
count  4717.000000  4708.000000  4698.000000    4698.000000   
mean     36.195588  -119.748634   784.361005   22366.834398   
std       2.465271     2.152344   490.071755   21262.234722   
min      32.555828  -124.301372     1.000000      11.000000   
25%      33.989524  -121.737696   349.000000    2550.750000   
50%      36.205465  -119.548156   757.000000   17846.000000   
75%      38.200231  -117.969795  1214.000000   36351.000000   
max      41.962127  -114.192901  1670.000000  105285.000000   

       Satisfaction Score  ...  Number of Referrals  Tenure in Months  \
count         4746.000000  ...          4733.000000       4723.000000   
mean             3.239992  ...             1.921403         32.615922   
std              1.204982  ...             2.979741         24.615757   
min              1.000000  ...             0.000000          1.000000   
25%              3.000000  ...             0.000000          9.000000   
50%              3.000000  ...             0.000000         29.000000   
75%              4.000000  ...             3.000000         56.000000   
max              5.000000  ...            11.000000         72.000000   

       Avg Monthly Long Distance Charges  Avg Monthly GB Download  \
count                        4722.000000              4731.000000   
mean                           22.841321                20.318537   
std                            15.478915                20.302725   
min                             0.000000                 0.000000   
25%                             9.052500                 3.000000   
50%                            22.385000                16.000000   
75%                            36.255000                27.000000   
max                            49.990000                85.000000   

       Monthly Charge  Total Charges  Total Refunds  Total Extra Data Charges  \
count     4734.000000    4732.000000    4739.000000               4739.000000   
mean        64.690906    2283.750864       1.814790                  6.490821   
std         30.191327    2255.975250       7.520257                 24.192935   
min         18.250000      18.850000       0.000000                  0.000000   
25%         34.800000     416.212500       0.000000                  0.000000   
50%         70.450000    1401.125000       0.000000                  0.000000   
75%         89.850000    3782.800000       0.000000                  0.000000   
max        118.750000    8672.450000      49.790000                150.000000   

       Total Long Distance Charges  Total Revenue  
count                  4724.000000    4744.000000  
mean                    747.372515    3038.374640  
std                     844.804363    2854.648396  
min                       0.000000      21.360000  
25%                      70.635000     622.862500  
50%                     399.390000    2114.875000  
75%                    1197.000000    4828.390000  
max                    3564.000000   11979.340000  

[8 rows x 21 columns]

In [365]:
for item in data.columns:
    print(item)
    print (data[item].unique())

Customer ID
['0650-BWOZN' '0562-FGDCR' '6688-UZPWD' ... '7649-PHJVR' '7855-DIWPO'
 '8197-BFWVU']
Count.x
[ 1. nan]
Gender
['Male' nan 'Female']
Age
[37. 54. nan 23. 41. 22. 33. 32. 21. 46. 49. 40. 47. 50. 64. 39. 52. 80.
 27. 65. 28. 63. 74. 57. 73. 77. 34. 72. 79. 58. 24. 29. 20. 66. 75. 25.
 67. 26. 31. 61. 62. 44. 35. 60. 51. 56. 70. 43. 68. 69. 45. 48. 78. 71.
 76. 55. 59. 38. 30. 42. 36. 19. 53.]
Under 30
['No' 'Yes' nan]
Senior Citizen
['No' nan 'Yes']
Married
[nan 'No' 'Yes']
Dependents
['No' 'Yes' nan]
Number of Dependents
[ 0. nan  2.  3.  1.  6.  4.  5.  7.  9.]
Count.y
[nan  1.]
Country
[nan 'United States']
State
[nan 'California']
City
[nan 'Rosamond' 'Huntington Beach' ... 'Coronado' 'Acton' 'Anderson']
Zip Code
[   nan 93560. 92649. ... 96091. 94708. 92532.]
Lat Long
[nan '34.903052, -118.411251' '33.721917, -118.043237' ...
 '37.327069, -121.916819' '34.559882, -115.637164'
 '33.688546, -117.788091']
Latitude
[      nan 34.903052 33.721917 ... 32.68674  37.327069 37.245

## 2.  Data Cleaning and Preprocessing

In [366]:
data.drop(['Customer ID', 'Country', 'State', 'City','Quarter',
           'Zip Code', 'Lat Long','Latitude','Longitude', 'Count','ID','Count.x', 'Count.y', 'Payment Method'], inplace=True, axis=1)

# we also drop county as it has only 1 value
data.shape

(7043, 37)

In [367]:
# checking the missing values and Handling them
# rows with all nan values
nan_rows = data[data.isnull().all(axis=1)]
nan


Int64Index([   6,   10,   19,   27,   39,   41,   42,   46,   49,   50,
            ...
            1358, 1361, 1365, 1368, 1372, 1375, 1377, 1395, 1399, 1408],
           dtype='int64', length=880)

In [368]:
# drop rows with all nan values
data.dropna(axis=0, thresh=5, subset=None, inplace=True)
data.shape

(6162, 37)

In [369]:
#Convert all yes and no to 0's & 1's so our classifier can use this data.

data["Gender"].replace(['Female','Male'],[0,1],inplace=True)
data["Married"].replace(['No', 'Yes'], [0, 1], inplace=True)
data["Under 30"].replace(['No', 'Yes'], [0, 1], inplace=True)
data["Senior Citizen"].replace(['No', 'Yes'], [0, 1], inplace=True)
data["Dependents"].replace(['No', 'Yes'], [0, 1], inplace=True)
data["Phone Service"].replace(['No', 'Yes'], [0, 1], inplace=True)
data["Paperless Billing"].replace(['No', 'Yes'], [0, 1], inplace=True)
data["Churn Category"].replace(['No Churn','Competitor', 
                                'Dissatisfaction', 'Other', 'Attitude' ,'Price'], [0, 1, 2, 3, 4, 5], inplace=True)
data["Streaming Movies"].replace(['No', 'Yes'], [0, 1], inplace=True)

data["Internet Type"].replace(['Fiber Optic', 'Cable', 'None', 'DSL'],[0,1,2, 3],inplace=True)

data["Contract"].replace(['One Year', 'Month-to-Month', 'Two Year'],[0,1,2],inplace=True)

#data = pd.get_dummies(data=data, columns=['Payment Method'])


#data["Payment Method"].replace(['Bank Withdrawal', 'Credit Card', 'Mailed Check'],[ 0, 1, 2], inplace=True)
data["Multiple Lines"].replace(['No','Yes'],[0,1],inplace=True)
data["Online Security"].replace(['No','Yes'],[0,1],inplace=True)
data["Online Backup"].replace(['No','Yes'],[0,1],inplace=True)
data["Device Protection Plan"].replace(['No','Yes'],[0,1],inplace=True)
data["Premium Tech Support"].replace(['No', 'Yes'], [0, 1], inplace=True)
data["Streaming TV"].replace(['No', 'Yes'], [0, 1], inplace=True)
data["Streaming Music"].replace(['No', 'Yes'], [0, 1], inplace=True)

data["Unlimited Data"].replace(['No', 'Yes'], [0, 1], inplace=True)
data["Internet Service"].replace(['No', 'Yes'], [0, 1], inplace=True)
data["Referred a Friend"].replace(['No', 'Yes'], [0, 1], inplace=True)


data["Offer"].replace(['None', 'Offer E', 'Offer C', 'Offer A' ,'Offer B', 'Offer D'],[ 0, 1, 2, 3, 4, 5], inplace=True)

data.head()


Gender   Age  Under 30  Senior Citizen  Married  Dependents  \
0     1.0  37.0       0.0             0.0      NaN         0.0   
1     NaN  54.0       0.0             0.0      0.0         0.0   
2     1.0   NaN       1.0             0.0      NaN         1.0   
3     1.0  23.0       1.0             0.0      0.0         0.0   
4     0.0  41.0       0.0             NaN      1.0         0.0   

   Number of Dependents  Population  Satisfaction Score  Referred a Friend  \
0                   0.0         NaN                 1.0                0.0   
1                   0.0     14931.0                 1.0                0.0   
2                   NaN     32304.0                 NaN                1.0   
3                   0.0     37570.0                 3.0                0.0   
4                   0.0       322.0                 NaN                1.0   

   ...  Contract  Paperless Billing  Monthly Charge  Total Charges  \
0  ...       0.0                1.0           85.15        3030.60   
1  ...       1.0                0.0           35.90          35.90   
2  ...       0.0                NaN           20.05         746.75   
3  ...       1.0                NaN           74.65         544.55   
4  ...       2.0                0.0           25.80        1911.50   

   Total Refunds  Total Extra Data Charges  Total Long Distance Charges  \
0            0.0                       0.0                       892.85   
1            0.0                       0.0                         0.00   
2            NaN                       0.0                      1435.70   
3            0.0                      10.0                          NaN   
4            0.0                       NaN                      1164.16   

   Total Revenue  Churn Category    ind  
0        3923.45             1.0  train  
1          35.90             1.0  train  
2        2182.45             0.0  train  
3         705.61             1.0  train  
4        3075.66             0.0  train  

[5 rows x 37 columns]

In [370]:
# Replace all nan values with predictive values
data.fillna(method='ffill', inplace=True)
data.tail()

Gender   Age  Under 30  Senior Citizen  Married  Dependents  \
1403     0.0  68.0       1.0             1.0      0.0         0.0   
1404     1.0  49.0       0.0             0.0      0.0         0.0   
1405     0.0  78.0       0.0             1.0      0.0         0.0   
1406     1.0  27.0       1.0             0.0      1.0         0.0   
1407     0.0  38.0       1.0             0.0      1.0         0.0   

      Number of Dependents  Population  Satisfaction Score  Referred a Friend  \
1403                   0.0       699.0                 3.0                0.0   
1404                   0.0     21205.0                 4.0                0.0   
1405                   0.0     21418.0                 4.0                0.0   
1406                   0.0     21418.0                 1.0                0.0   
1407                   3.0     27369.0                 1.0                1.0   

      ...  Contract  Paperless Billing  Monthly Charge  Total Charges  \
1403  ...       1.0                0.0           94.45        1511.20   
1404  ...       0.0                0.0           94.45         304.60   
1405  ...       0.0                0.0           94.45         304.60   
1406  ...       1.0                1.0          106.10        2249.95   
1407  ...       0.0                0.0           19.90         533.50   

      Total Refunds  Total Extra Data Charges  Total Long Distance Charges  \
1403            0.0                       0.0                       150.88   
1404            0.0                       0.0                        83.10   
1405            0.0                       0.0                        83.10   
1406            0.0                       0.0                       458.43   
1407            0.0                       0.0                       458.43   

      Total Revenue  Churn Category   ind  
1403        1662.08             0.0  test  
1404         387.70             0.0  test  
1405         387.70             0.0  test  
1406         387.70             0.0  test  
1407         640.78             0.0  test  

[5 rows x 37 columns]

In [371]:

data.isnull().values.any()  # data still contains some nan values, need to be removed

True

In [372]:
# Our data is clean and now we can split back to test and train before Training the model

train_set, test_set = data[data["ind"].eq("train")], data[data["ind"].eq("test")]

X_train = train_set.loc[:,'Gender': 'Total Revenue']
y_train = train_set["Churn Category"]


X_test = test_set.loc[:,'Gender': 'Total Revenue']
y_test = test_set["Churn Category"]

print(X_test.shape)
print(y_train.shape)

(1226, 35)
(4936,)
